In [21]:
import simulation
from quality_measures import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

SIMULATION_PARAMETERS = {
    "N_YEARS": 1,
    "N_USERS_START": 1000,
    "N_NEW_USERS_PER_YEAR": 50,
    "P_USER_EXITS_PER_YEAR": 0.1,
    "N_CONTENT_PER_USER_PER_YEAR": 1,
    "N_REVIEWS_PER_USER_PER_YEAR": 3,
    "N_REVIEW_SCORES_PER_USER_PER_YEAR": 10
}

In [ ]:
measures = [SimpleMean(), SimpleMeanThresholdedReviewers(50), SimpleMeanThresholdedReviewers(80), BayesWeightingOracle()]
measures.append(BayesWeightingMeasureEstimate(measures[2], 0.1, 0.1))
measures.append(BayesWeightingMeasureEstimate(measures[2], 0.1, 0.1, threshold_percentile=20))
measure_performances = [[] for measure in measures]

commitment_resolution = 0.05
N_TRIALS = 10

for trial_i in range(N_TRIALS):
    platform = simulation.run(SIMULATION_PARAMETERS=SIMULATION_PARAMETERS)
    for measure_i, measure in enumerate(measures):
        measure.calculate_estimates(platform)
        measure_performances[measure_i].append(measure.evaluate_performance(platform, commitment_resolution=commitment_resolution))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

measure_colors = cm.hsv(np.linspace(0, .9, len(measures)))

fig, ax1 = plt.subplots(1, 1, figsize=(4, 4))
plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})

for measure_i, measure in enumerate(measures):
    # Find the maximum length of the trials for the current measure
    max_length = max(len(measure_performances[measure_i][trial_i][2]) for trial_i in range(N_TRIALS))
    
    # Create arrays to store the interpolated data for each trial
    interpolated_x = np.zeros((N_TRIALS, max_length))
    interpolated_y = np.zeros((N_TRIALS, max_length))
    interpolated_x[:] = np.nan
    interpolated_y[:] = np.nan
    
    for trial_i in range(N_TRIALS):
        # Interpolate the data to match the maximum length
        x = measure_performances[measure_i][trial_i][2]
        y = measure_performances[measure_i][trial_i][3]
        interpolated_x[trial_i, :len(x)] = x
        interpolated_y[trial_i, :len(x)] = y
    
    # Calculate the mean and standard deviation across trials
    mean_performance = np.mean(interpolated_y, axis=0)
    std_performance = np.std(interpolated_y, axis=0)
    
    # Plot the mean performance with a shaded region representing the standard deviation
    x_values = np.mean(interpolated_x, axis=0)
    ax1.plot(x_values, mean_performance, "-", lw=4, color=measure_colors[measure_i], label=measure.name)
    ax1.fill_between(x_values, mean_performance - std_performance, mean_performance + std_performance,
                     color=measure_colors[measure_i], alpha=0.3)

ax1.set_xlim(0, 1)
ax1.set_ylim(.5, 1)
ax1.set_xlabel("commitment")
ax1.set_ylabel("correlation")
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))


plt.show()